In [ ]:
from pathlib import Path
import os

from mozaik.tools.distribution_parametrization import load_parameters

In [ ]:
def flatten_dict(d, parent_key='', sep='.'):
    """
    Flatten a nested dictionary.
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def compare_parameters(param1, param2):
    """
    Compare two ParameterSet objects and return differences.
    """
    flat1 = flatten_dict(param1.to_dict())
    flat2 = flatten_dict(param2.to_dict())
    
    differences = {}
    for key in set(flat1.keys()).union(flat2.keys()):
        if "recorders" in key:
            continue
        if flat1.get(key) != flat2.get(key):
            differences[key] = (flat1.get(key), flat2.get(key))
    
    return differences

In [ ]:
# Pick model

# model_name = 'LSV1M_split_full'

model_name = 'LSV1M_split_ee'

# model_name = 'LSV1M_tsodyks2'

# model_name = 'LSV1M_split_remy'

# model_name = 'LSV1M_refs-update'

# model_name = 'LSV1M__infragranular'


In [ ]:
# Part I: Compare parameters of one model with different recorders

model_path = Path(f'/home/haman/layers56/{model_name}/')
os.chdir(model_path)

################################################################################

print("Comparing LOADED parameters:")

param = load_parameters("param/defaults",{})
param_spont = load_parameters("param_spont/defaults", {})

try:
    param_optim = load_parameters("param_optim/defaults", {})
except FileNotFoundError:
    print("No 'param_optim' found, skipping optimization parameters.")
    param_optim = None

print(f"Loaded parameters from {model_path}")

print('Differences between "param"       and  "param_spont": ', compare_parameters(param, param_spont))
if param_optim:
    print('Differences between "param"       and  "param_optim": ', compare_parameters(param, param_optim))
    print('Differences between "param_spont" and  "param_optim": ', compare_parameters(param_spont, param_optim))
print()

################################################################################

print("Comparing parameter files:")

file_list = [
    "defaults",
    "l4_cortex_exc",
    "l4_cortex_inh",
    "l23_cortex_exc",
    "l23_cortex_inh",
    "SpatioTemporalFilterRetinaLGN_defaults"
]

if param_optim:
    for file_name in file_list:
        print(f"\nComparing {file_name}\n")
        with (open(f"param/{file_name}", 'r') as f1, 
                    open(f"param_optim/{file_name}", 'r') as f2, 
                    open(f"param_spont/{file_name}", 'r') as f3):
            line_num = 0
            for line1, line2, line3 in zip(f1, f2, f3):
                line_num += 1
                if line1 != line2 or line1 != line3 or line2 != line3:
                    print(f"--- Line {line_num} ---")
                    print(f"File 1: {line1.strip()}")
                    print(f"File 2: {line2.strip()}")
                    print(f"File 3: {line3.strip()}")
else:
    for file_name in file_list:
        print(f"\nComparing {file_name}\n")
        with (open(f"param/{file_name}", 'r') as f1, 
                    open(f"param_spont/{file_name}", 'r') as f2):
            line_num = 0
            for line1, line2 in zip(f1, f2):
                line_num += 1
                if line1 != line2:
                    print(f"--- Line {line_num} ---")
                    print(f"File 1: {line1.strip()}")
                    print(f"File 2: {line2.strip()}")

Comparing LOADED parameters:
Loaded parameters from /home/haman/layers56/LSV1M_split_ee
Differences between "param"       and  "param_spont":  {}
Differences between "param"       and  "param_optim":  {}
Differences between "param_spont" and  "param_optim":  {}

Comparing parameter files:

Comparing defaults

--- Line 12 ---
File 1: 'l4_cortex_exc': url("param/l4_cortex_exc"),
File 2: 'l4_cortex_exc': url("param_optim/l4_cortex_exc"),
File 3: 'l4_cortex_exc': url("param_spont/l4_cortex_exc"),
--- Line 13 ---
File 1: 'l4_cortex_inh': url("param/l4_cortex_inh"),
File 2: 'l4_cortex_inh': url("param_optim/l4_cortex_inh"),
File 3: 'l4_cortex_inh': url("param_spont/l4_cortex_inh"),
--- Line 14 ---
File 1: 'l23_cortex_exc': url("param/l23_cortex_exc"),
File 2: 'l23_cortex_exc': url("param_optim/l23_cortex_exc"),
File 3: 'l23_cortex_exc': url("param_spont/l23_cortex_exc"),
--- Line 15 ---
File 1: 'l23_cortex_inh': url("param/l23_cortex_inh"),
File 2: 'l23_cortex_inh': url("param_optim/l23_cort

In [69]:
# Part II: Compare parameters of two models

model_name1 = 'LSV1M_tsodyks2'
model_name2 = 'LSV1M_refs-update'

expected_differences = {}
# expected_differences = {"L23ExcL23InhConnectionLong", "L23ExcL23InhConnectionShort"}#, "L23ExcL23InhConnection"}

################################################################################

print("Comparing LOADED parameters:")

model_path1 = Path(f'/home/haman/layers56/{model_name1}/')
os.chdir(model_path1)
param1 = load_parameters("param/defaults",{})

model_path2 = Path(f'/home/haman/layers56/{model_name2}/')
os.chdir(model_path2)
param2 = load_parameters("param/defaults",{})

print(f'Differences between models: "{model_name1}" and "{model_name2}": ')
for key, (value1, value2) in compare_parameters(param1, param2).items():
    if any(difference in key for difference in expected_differences):
        continue
    else:
        print(key, ":", value1, value2)


Comparing LOADED parameters:
Differences between models: "LSV1M_tsodyks2" and "LSV1M_refs-update": 
sheets.l4_cortex_inh.L4InhL4InhConnection.short_term_plasticity.tau_psc : None 3.0
sheets.l4_cortex_exc.AfferentConnection.short_term_plasticity.tau_psc : None 3.0
sheets.l4_cortex_inh.L4InhL4ExcConnection.short_term_plasticity.tau_psc : None 3.0
sheets.l23_cortex_exc.L23ExcL4ExcConnection.short_term_plasticity.tau_psc : None 3.0
sheets.l4_cortex_exc.L4ExcL4InhConnection.short_term_plasticity.tau_psc : None 3.0
sheets.l23_cortex_inh.L23InhL23ExcConnection.short_term_plasticity.tau_psc : None 3.0
sheets.l23_cortex_exc.L23ExcL4InhConnection.short_term_plasticity.tau_psc : None 3.0
sheets.l23_cortex_exc.L23ExcL4InhConnection.delay_functions.d1.params.constant_scaler : 1.4 0.5
sheets.l23_cortex_exc.L4ExcL23ExcConnection.short_term_plasticity.tau_psc : None 3.0
sheets.l23_cortex_exc.L23ExcL23InhConnection.short_term_plasticity.tau_psc : None 3.0
sheets.l23_cortex_inh.L4ExcL23InhConnection.sho

In [ ]:
# Part III: pick param folder and make copy of it with changed recorders


